# Text to Audio

1. Prompt user for advertisement description
2. Perform sentimental analysis
3. Add the corresponding sentimental analysis into the user prompt
4. Generate the audio for the advertisement

In [21]:
# Ensure relevant libraries are installed

!pip install transformers
!pip install scipy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


### Sentimental Analysis

In [22]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

# Input from user is stored into userInput
userInput = input("Enter your ideal advertisement description: ")

# analysis stores the classified user input for with the different confidence level of the emotions detected
analysis = classifier(userInput)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/Users/shuyi/Documents/NTU/Y3S1/AI/LSVD/venv/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [23]:
analysis

[[{'label': 'anger', 'score': 0.0010561796370893717},
  {'label': 'disgust', 'score': 0.0009741616086103022},
  {'label': 'fear', 'score': 0.002229639096185565},
  {'label': 'joy', 'score': 0.03577585518360138},
  {'label': 'neutral', 'score': 0.005105865653604269},
  {'label': 'sadness', 'score': 0.9249907732009888},
  {'label': 'surprise', 'score': 0.02986745908856392}]]

Filtering of emotions to be added into the userInput, set a confidence level of minimum 0.40 to be considered as part of the mood in the advertisement.

In [24]:
filtered_analysis = sorted([item for item in analysis[0] if item['score'] > 0.4], key=lambda x: x['score'], reverse=True)

filtered_analysis

[{'label': 'sadness', 'score': 0.9249907732009888}]

In [25]:
labels_string = "mood: " + ", ".join([item['label'] for item in filtered_analysis])
print(labels_string)

mood: sadness


Final user input to be used to generate the audio

In [26]:
edited_user_input = "\n".join([userInput, labels_string])
print(edited_user_input)

Happy woman walking down the street and saw a lonely cat
mood: sadness


### Audio Generation from User Input

In [27]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

inputs = processor(
    text=[edited_user_input],
    padding=True,
    return_tensors="pt",
)

audio_values = model.generate(**inputs, max_new_tokens=400)

/Users/shuyi/Documents/NTU/Y3S1/AI/LSVD/venv/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [28]:
from IPython.display import Audio

sampling_rate = model.config.audio_encoder.sampling_rate
Audio(audio_values[0].numpy(), rate=sampling_rate)

In [29]:
import scipy

sampling_rate = model.config.audio_encoder.sampling_rate
scipy.io.wavfile.write("musicgen_out.wav", rate=sampling_rate, data=audio_values[0, 0].numpy())